Lab 13: Implement a Basic Artificial Neural Network
This script demonstrates implementation of a basic ANN using TensorFlow/Keras.

In [9]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits, load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

# Try to import tensorflow/keras, provide fallback message if not available
try:
    from tensorflow import keras
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout
    from tensorflow.keras.utils import to_categorical
    KERAS_AVAILABLE = True
except ImportError:
    KERAS_AVAILABLE = False
    print("TensorFlow/Keras not available. Install with: pip install tensorflow")


In [10]:
def simple_neural_network():
    """Demonstrate a simple neural network for classification"""
    if not KERAS_AVAILABLE:
        print("\nSkipping: TensorFlow/Keras required")
        return

    print("=" * 50)
    print("Simple Neural Network")
    print("=" * 50)

    # Load dataset
    iris = load_iris()
    X = iris.data
    y = iris.target

    print(f"\nDataset: Iris")
    print(f"Input shape: {X.shape}")
    print(f"Number of classes: {len(np.unique(y))}")

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Standardize features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Convert labels to categorical
    y_train_cat = to_categorical(y_train, num_classes=3)
    y_test_cat = to_categorical(y_test, num_classes=3)

    # Create neural network
    model = Sequential([
        Dense(8, activation='relu', input_shape=(4,)),
        Dense(6, activation='relu'),
        Dense(3, activation='softmax')
    ])

    # Compile model
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print("\nModel Architecture:")
    model.summary()

    # Train model
    print("\nTraining model...")
    history = model.fit(
        X_train_scaled, y_train_cat,
        epochs=100,
        batch_size=16,
        validation_split=0.2,
        verbose=0
    )

    # Evaluate
    test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test_cat, verbose=0)
    print(f"Test Loss: {test_loss:.4f}")

    # Make predictions
    y_pred_proba = model.predict(X_test_scaled, verbose=0)
    y_pred = np.argmax(y_pred_proba, axis=1)

    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=iris.target_names))

    # Plot training history
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Model Accuracy')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Model Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('lab13_training_history.png')
    plt.close()
    print("\nTraining history saved as 'lab13_training_history.png'")

    return model, history


In [11]:
def deep_neural_network():
    """Demonstrate a deeper neural network"""
    if not KERAS_AVAILABLE:
        print("\nSkipping: TensorFlow/Keras required")
        return

    print("\n" + "=" * 50)
    print("Deep Neural Network with Dropout")
    print("=" * 50)

    # Load digits dataset
    digits = load_digits()
    X = digits.data
    y = digits.target

    print(f"\nDataset: Handwritten Digits")
    print(f"Input shape: {X.shape}")
    print(f"Number of classes: {len(np.unique(y))}")

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Normalize features
    X_train_norm = X_train / 16.0
    X_test_norm = X_test / 16.0

    # Convert labels to categorical
    y_train_cat = to_categorical(y_train, num_classes=10)
    y_test_cat = to_categorical(y_test, num_classes=10)

    # Create deep neural network with dropout
    model = Sequential([
        Dense(128, activation='relu', input_shape=(64,)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(10, activation='softmax')
    ])

    # Compile model
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print("\nModel Architecture:")
    model.summary()

    # Train model
    print("\nTraining model...")
    history = model.fit(
        X_train_norm, y_train_cat,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        verbose=0
    )

    # Evaluate
    test_loss, test_accuracy = model.evaluate(X_test_norm, y_test_cat, verbose=0)
    print(f"Test Loss: {test_loss:.4f}")

    # Make predictions
    y_pred_proba = model.predict(X_test_norm, verbose=0)
    y_pred = np.argmax(y_pred_proba, axis=1)

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix - Deep Neural Network')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig('lab13_deep_nn_confusion.png')
    plt.close()
    print("\nConfusion matrix saved as 'lab13_deep_nn_confusion.png'")


In [12]:
def activation_functions_comparison():
    """Compare different activation functions"""
    if not KERAS_AVAILABLE:
        print("\nSkipping: TensorFlow/Keras required")
        return

    print("\n" + "=" * 50)
    print("Comparing Activation Functions")
    print("=" * 50)

    # Load dataset
    iris = load_iris()
    X = iris.data
    y = iris.target

    # Prepare data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    y_train_cat = to_categorical(y_train, num_classes=3)
    y_test_cat = to_categorical(y_test, num_classes=3)

    # Test different activation functions
    activations = ['relu', 'tanh', 'sigmoid']
    results = {}

    for activation in activations:
        model = Sequential([
            Dense(8, activation=activation, input_shape=(4,)),
            Dense(6, activation=activation),
            Dense(3, activation='softmax')
        ])

        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        history = model.fit(
            X_train_scaled, y_train_cat,
            epochs=100,
            batch_size=16,
            validation_split=0.2,
            verbose=0
        )

        _, accuracy = model.evaluate(X_test_scaled, y_test_cat, verbose=0)
        results[activation] = accuracy


    # Visualize comparison
    plt.figure(figsize=(10, 6))
    plt.bar(results.keys(), results.values(), color=['blue', 'green', 'orange'])
    plt.ylabel('Test Accuracy')
    plt.title('Comparison of Activation Functions')
    plt.ylim([0.9, 1.0])
    for i, (act, acc) in enumerate(results.items()):
        plt.text(i, acc + 0.005, f'{acc:.4f}', ha='center')
    plt.tight_layout()
    plt.savefig('lab13_activation_comparison.png')
    plt.close()
    print("\nActivation comparison saved as 'lab13_activation_comparison.png'")


In [13]:
def visualize_weights():
    """Visualize neural network weights"""
    if not KERAS_AVAILABLE:
        print("\nSkipping: TensorFlow/Keras required")
        return

    print("\n" + "=" * 50)
    print("Visualizing Network Weights")
    print("=" * 50)

    # Create and train a simple model
    iris = load_iris()
    X = iris.data
    y = iris.target

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    y_cat = to_categorical(y, num_classes=3)

    model = Sequential([
        Dense(4, activation='relu', input_shape=(4,)),
        Dense(3, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_scaled, y_cat, epochs=50, verbose=0)

    # Get weights
    weights = model.get_weights()

    print(f"\nLayer 1 weights shape: {weights[0].shape}")
    print(f"Layer 1 bias shape: {weights[1].shape}")
    print(f"Layer 2 weights shape: {weights[2].shape}")
    print(f"Layer 2 bias shape: {weights[3].shape}")

    # Visualize first layer weights
    plt.figure(figsize=(10, 6))
    sns.heatmap(weights[0], cmap='coolwarm', center=0,
                xticklabels=[f'Neuron {i+1}' for i in range(4)],
                yticklabels=iris.feature_names)
    plt.title('First Layer Weights')
    plt.xlabel('Hidden Layer Neurons')
    plt.ylabel('Input Features')
    plt.tight_layout()
    plt.savefig('lab13_weights_visualization.png')
    plt.close()
    print("\nWeights visualization saved as 'lab13_weights_visualization.png'")


In [14]:
def manual_neural_network():
    """Implement a simple neural network manually"""
    print("\n" + "=" * 50)
    print("Manual Neural Network Implementation")
    print("=" * 50)

    # Activation functions
    def sigmoid(x):
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

    def sigmoid_derivative(x):
        return x * (1 - x)

    # Generate simple XOR dataset
    X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
    y = np.array([[0], [1], [1], [0]])

    print("\nTraining on XOR problem:")
    print("Input:\n", X)
    print("Expected Output:\n", y)

    # Initialize weights
    np.random.seed(42)
    input_neurons = 2
    hidden_neurons = 4
    output_neurons = 1

    weights_input_hidden = np.random.uniform(-1, 1, (input_neurons, hidden_neurons))
    weights_hidden_output = np.random.uniform(-1, 1, (hidden_neurons, output_neurons))

    learning_rate = 0.5
    epochs = 10000

    # Training
    errors = []
    for epoch in range(epochs):
        # Forward propagation
        hidden_input = np.dot(X, weights_input_hidden)
        hidden_output = sigmoid(hidden_input)

        final_input = np.dot(hidden_output, weights_hidden_output)
        final_output = sigmoid(final_input)

        # Calculate error
        error = y - final_output
        errors.append(np.mean(np.abs(error)))

        # Backpropagation
        d_output = error * sigmoid_derivative(final_output)
        error_hidden = d_output.dot(weights_hidden_output.T)
        d_hidden = error_hidden * sigmoid_derivative(hidden_output)

        # Update weights
        weights_hidden_output += hidden_output.T.dot(d_output) * learning_rate
        weights_input_hidden += X.T.dot(d_hidden) * learning_rate

    print(f"\nTraining complete after {epochs} epochs")
    print(f"Final error: {errors[-1]:.6f}")

    print("\nPredictions:")
    for i in range(len(X)):
        hidden = sigmoid(np.dot(X[i], weights_input_hidden))
        output = sigmoid(np.dot(hidden, weights_hidden_output))
        print(f"Input: {X[i]} → Output: {output[0]:.4f} (Expected: {y[i][0]})")

    # Plot error over epochs
    plt.figure(figsize=(10, 6))
    plt.plot(errors)
    plt.xlabel('Epoch')
    plt.ylabel('Mean Absolute Error')
    plt.title('Training Error Over Time (Manual NN)')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('lab13_manual_nn_training.png')
    plt.close()
    print("\nManual NN training plot saved as 'lab13_manual_nn_training.png'")


In [15]:
def main():
    """Main function to demonstrate artificial neural networks"""
    print("\n" + "=" * 50)
    print("Lab 13: Artificial Neural Network")
    print("=" * 50)

    # Simple neural network
    simple_neural_network()

    # Deep neural network
    deep_neural_network()

    # Activation functions comparison
    activation_functions_comparison()

    # Visualize weights
    visualize_weights()

    # Manual implementation
    manual_neural_network()

    print("\n" + "=" * 50)
    print("Lab 13 Complete!")
    print("=" * 50)


In [16]:
if __name__ == "__main__":
    main()



Lab 13: Artificial Neural Network
Simple Neural Network

Dataset: Iris
Input shape: (150, 4)
Number of classes: 3

Model Architecture:


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (None, 8)              │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 6)              │            54 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 3)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 115 (460.00 B)

...


Training model...
Test Loss: 0.1091

Confusion matrix saved as 'lab13_deep_nn_confusion.png'

Comparing Activation Functions


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Activation comparison saved as 'lab13_activation_comparison.png'

Visualizing Network Weights


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Layer 1 weights shape: (4, 4)
Layer 1 bias shape: (4,)
Layer 2 weights shape: (4, 3)
Layer 2 bias shape: (3,)

Weights visualization saved as 'lab13_weights_visualization.png'

Manual Neural Network Implementation

Training on XOR problem:
Input:
 [[0 0]
 [0 1]
 [1 0]
 [1 1]]
Expected Output:
 [[0]
 [1]
 [1]
 [0]]

Training complete after 10000 epochs
Final error: 0.022058

Predictions:
Input: [0 0] → Output: 0.0277 (Expected: 0)
Input: [0 1] → Output: 0.9855 (Expected: 1)
Input: [1 0] → Output: 0.9747 (Expected: 1)
Input: [1 1] → Output: 0.0207 (Expected: 0)

Manual NN training plot saved as 'lab13_manual_nn_training.png'

Lab 13 Complete!
